In [1]:
import numpy as np
import torch

In [2]:
!pip install tensorly

In [16]:
def quantization_forw(T,z,b,scale):
  T = np.clip(np.round(T/scale)+z,-2**(b-1),2**(b-1)-1)
  return T.int()

def quantization_back(T,z,scale):
  return scale * (T - z)

def ADMM(B, U, K, G, rank, scale):
    ro = torch.trace(G) / rank

    LL = G + ro * torch.eye(*G.size())
    L = torch.linalg.cholesky(LL)
    LL_inv = torch.cholesky_inverse(L)

    r = torch.inf
    s = torch.inf

    for _ in range(100):
    # while (r > 0.01) or (s > 0.01):
        # print(r, s)
        B_ = LL_inv @ (K + ro * (B + U)).T
        B0 = torch.clone(B)

        B = quantization_back(quantization_forw(B_.T - U, 0, 8, scale), 0, scale)
        U = U + B - B_.T

        r = torch.norm(B - B_.T, p='fro')**2 / torch.norm(B, p='fro')**2
        s = torch.norm(B - B0, p='fro')**2 / torch.norm(U, p='fro')**2
    return B, U

In [40]:
from tensorly.decomposition import parafac

def e_quant(X,A,B):
  return torch.norm(X - A@B.T, p='fro')/torch.norm(X, p='fro')

M = 6
N = 8

X = np.random.rand(M, N)*10
reduction_rate = 2
rank = round(M*N/(M + N)/reduction_rate)

weights, factors = parafac(X, rank=rank)
A = torch.from_numpy(factors[0])
B = torch.from_numpy(factors[1])

print(A@B.T)

X = torch.from_numpy(X)

U_A = torch.zeros_like(A)
U_B = torch.zeros_like(B)

e_old = 0.
e_new = e_quant(X,A,B)

b=8
#scale = (torch.max(X)-torch.min(X))/(2**b-1)
q_max = 9.89675982 * torch.mean(torch.abs(X - X.mean()))
scale = 2*q_max/(2**b-1)

while torch.abs(e_new - e_old) > 0.01:
    print(torch.abs(e_new - e_old))
    K = X.T @ A
    G = A.T @ A
    B, U_B = ADMM(B, U_B, K, G, rank, scale)

    K = X @ B
    G = B.T @ B
    A, U_A = ADMM(A, U_A, K, G, rank, scale)

    e_old = e_new
    e_new = e_quant(X,A,B)

tensor([[ 6.8027,  7.8547,  8.5403,  8.2032,  4.5853, 10.4625,  4.7139,  6.6861],
        [ 1.6585,  5.4313,  2.8276,  6.6951,  2.5012,  5.7220,  4.2487,  5.6129],
        [ 1.3603,  5.4062,  2.5209,  6.7618,  2.4258,  5.5513,  4.3235,  5.6815],
        [ 4.1776,  7.1743,  5.7430,  8.1764,  3.7407,  8.5451,  4.9669,  6.7686],
        [ 5.1084,  3.9433,  5.9987,  3.5495,  2.6741,  6.0934,  1.8166,  2.8063],
        [ 8.4806,  3.6342,  9.3413,  2.0041,  3.2936,  7.4893,  0.4487,  1.3601]],
       dtype=torch.float64)
tensor(0.3400, dtype=torch.float64)
tensor(0.0234, dtype=torch.float64)


In [41]:
A@B.T

tensor([[ 5.4471,  8.3955,  9.8947,  7.8958,  4.4476,  8.8952,  3.9479,  7.8958],
        [ 1.1994,  5.6470,  3.6980,  6.2966,  2.4987,  4.9973,  3.1483,  6.2966],
        [ 0.5997,  5.6969,  2.9984,  6.5965,  2.3987,  4.7974,  3.2982,  6.5965],
        [ 3.4981,  7.4960,  7.1961,  7.5959,  3.6980,  7.3960,  3.7980,  7.5959],
        [ 4.4976,  4.2477,  7.0962,  3.2982,  2.5986,  5.1972,  1.6491,  3.2982],
        [ 7.5460,  3.8479, 10.5943,  1.5991,  3.0484,  6.0967,  0.7996,  1.5991]],
       dtype=torch.float64)

In [20]:
from tensorly.decomposition import parafac

def e_quant(X,A,B):
  return torch.norm(X - A@B.T, p='fro')/torch.norm(X, p='fro')

def factorize(W, reduction_rate = 2):
    M,N = W.shape
    rank = round(M*N/(M + N)/reduction_rate)

    #W - должен быть просто numpy массивом
    weights, factors = parafac(W, rank=rank)
    A = torch.from_numpy(factors[0])
    B = torch.from_numpy(factors[1])
    new_W = A@B.T

    W = torch.from_numpy(W)

    U_A = torch.zeros_like(A)
    U_B = torch.zeros_like(B)

    e_old = 0.
    e_new = e_quant(W,A,B)

    b=8
    #scale1 = (torch.max(A)-torch.min(A))/(2**b-1)
    #scale2 = (torch.max(B)-torch.min(B))/(2**b-1)
    #q_max = 9.89675982 * torch.mean(torch.abs(A - A.mean()))
    #scale2 = 2*q_max/(2**b-1)
    #q_max = 9.89675982 * torch.mean(torch.abs(B - B.mean()))
    #scale1 = 2*q_max/(2**b-1)
    q_max = 9.89675982 * torch.mean(torch.abs(W - W.mean()))
    # scale = 2*q_max/(2**b-1)

    scale = (torch.max(torch.abs(W))/(2**(8-1)-1))

    while torch.abs(e_new - e_old) > 0.01:
        print(f'error: {torch.abs(e_new - e_old)}')
        K = W @ B
        G = B.T @ B
        A, U_A = ADMM(A, U_A, K, G, rank, scale)  
        
        K = W.T @ A
        G = A.T @ A
        B, U_B = ADMM(B, U_B, K, G, rank, scale)

        e_old = e_new
        e_new = e_quant(X,A,B)
    
    print(f'error: {torch.abs(e_new - e_old)}')

    return new_W, A, B, scale

In [21]:
#W - матрица весов в .numpy
X = np.random.rand(6, 8)*10
X = torch.from_numpy(X)
newX, A, B, scale = factorize(X.numpy())

error: 0.3163478172648995
error: 0.024230089950282474
error: 0.011770858379707316
error: 0.005675450387603698


In [22]:
X.numpy(), newX, A@B.T, scale

(array([[6.72747202, 6.10906426, 8.52791918, 9.92261306, 9.94754311,
         2.30469858, 5.25550473, 2.69578374],
        [3.5871028 , 9.23017548, 9.21799721, 6.45569961, 2.54199481,
         3.0045621 , 1.21453183, 2.19483124],
        [7.90550384, 3.48360502, 2.27560604, 3.69690455, 8.29237368,
         2.67094683, 1.08367524, 7.09211689],
        [1.97623797, 6.11291657, 9.38756413, 5.00168264, 3.92483548,
         7.87870521, 4.7067804 , 5.1387224 ],
        [8.83665428, 4.68968889, 3.24781985, 9.05458534, 2.14555517,
         9.85939397, 3.72208043, 7.86031403],
        [4.37589395, 0.09990548, 9.71908443, 9.55140093, 6.84323541,
         0.47116207, 4.87860879, 3.18443112]]),
 tensor([[ 5.5807,  5.6169, 10.6924,  9.6252,  8.1083,  2.8784,  4.8354,  3.7958],
         [ 4.4905,  4.2645,  7.2391,  6.9226,  5.5884,  2.8983,  3.4286,  3.3578],
         [ 5.3683,  4.2791,  4.2739,  5.6362,  3.6764,  5.3339,  2.6154,  4.9885],
         [ 5.7599,  4.9470,  6.4884,  7.1940,  5.2497,  4.9

In [31]:
A_int = (A/scale).int()
B_int = (B/scale).int()

In [33]:
A_int@B_int.T * scale**2

tensor([[ 4.8836,  5.1167,  9.8162,  8.4665,  7.5830,  2.7547,  4.2332,  3.7670],
        [ 4.7609,  4.5339,  7.5708,  6.9818,  5.9388,  3.5032,  3.4909,  3.9449],
        [ 5.4971,  4.2885,  4.5768,  5.4112,  3.8283,  5.7486,  2.7056,  5.1228],
        [ 6.0615,  5.1044,  6.6996,  7.0186,  5.4235,  5.6627,  3.5093,  5.4235],
        [ 8.8101,  6.0983,  3.9265,  6.5032,  3.6565, 10.6077,  3.2516,  8.6751],
        [ 3.7792,  4.4725,  9.8530,  7.9879,  7.5094,  1.2086,  3.9940,  2.6074]],
       dtype=torch.float64)

In [35]:
#придумай свой входной вектор
Z = torch.from_numpy(np.array([2.,5.,4.,4.,2.,1.]))
Z@X

tensor([ 92.9666, 106.2344, 126.0132, 114.5786,  92.6082,  82.0208,  52.0683,
         84.1941], dtype=torch.float64)

In [39]:
scale_Z = (torch.max(torch.abs(Z))/(2**(8-1)-1))
Z_int = quantization_forw(Z, 0, 8, scale_Z)

In [41]:
(Z_int @ A_int @ B_int.T) * scale_Z * scale**2

tensor([101.4355,  87.3093, 120.4284, 122.7433,  96.8057,  91.3604,  61.3717,
         89.6242], dtype=torch.float64)